# Facemap Pose Estimation -- Run Inference Notebook

In [ ]:
import datetime
import datajoint as dj
import os

# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd()) == "notebooks":
    os.chdir("..")
dj.config.load('dj_local_conf.json')

from workflow.pipeline import *
from workflow.utils.ingest import ingest_model, generate_facemap_pose_estimation_task

#### Insert Subject and Session into subject.Subject, session.Session and session.SessionDirectory tables

In [ ]:
sub_insert = dict(subject="mdl_sub", 
                subject_nickname="facemap model subject", 
                sex='U', 
                subject_birth_date=datetime.datetime.now(), 
                subject_description="Subject for Facemap Model Inference testing")
# subject.Subject.insert1(sub_insert)
subject_key = (subject.Subject & 'subject="mdl_sub"').fetch1("KEY")

In [ ]:
session_id = 2
session_insert = dict(subject_key, session_id, session_datetime=datetime.datetime.now())
sdir_insert = dict(subject_key, session_id, session_dir="20230627_Image_eCBsensor_activity/Behavior_20230627/C57-C11-3_Rm_CNO_30min")

session.Session.insert1(session_insert)
session.SessionDirectory.insert1(sdir_insert)

In [ ]:
# Display Session Table to validate insert
session.Session() & {**subject_key, 'session_id': session_id}

In [ ]:
# Display SessionDirectory Table to validate insert
session.SessionDirectory() & {**subject_key, 'session_id': session_id}

### Ingest locally stored pytorch model(.pt) file
Provide model name, model filepath, and optional model description

In [ ]:
model_name = 'facemap_model_state.pt'
full_local_model_filepath = "/Users/sidhulyalkar/.facemap/models/facemap_model_state.pt"
ingest_model(model_name, model_description="test facemap model", model_file=full_local_model_filepath)

### Generate a Pose Estimation Task

In [ ]:
model_id = 0
session_key = session.Session.fetch("KEY")[2] 
generate_facemap_pose_estimation_task(model_id, session_key, task_mode="trigger", bbox=[])

#### Display FacemapPoseEstimationTask table

In [ ]:
facemap_pose.FacemapPoseEstimationTask()

#### Display VideoRecording and VideoRecording.File tables from the imported facial behavioral estimation (fbe) schema

In [ ]:
fbe.VideoRecording()

In [ ]:
fbe.VideoRecording.File()

#### Run Pose Estimation on all unprocessed FacemapPoseEstimationTasks  

In [ ]:
facemap_pose.FacemapPoseEstimation.populate(display_progress=True)
# If a lost connection error occurs, rerun the populate and if processing 
# has completed, the data will be loaded and inference will not be rerun. 


#### Display Facemap Pose Estimation Tables

In [ ]:
facemap_pose.FacemapPoseEstimation()

In [ ]:
facemap_pose.FacemapPoseEstimation.BodyPartPosition()

#### Visualize Pose Estimation Output

In [ ]:
pe_query = {**session_key, 'recording_id': 0, 'model_id': model_id}
pose_estimation_key = (facemap_pose.FacemapPoseEstimation & pe_query).fetch1("KEY")

Get Trajectory of X and Y coordinates

In [ ]:
# Specify all body parts, or set body_parts to a custom list
body_parts = "all"
model_name = (facemap_pose.FacemapModel & f'model_id={key["model_id"]}').fetch1("model_name")

if body_parts == "all":
    body_parts = (facemap_pose.BodyPartPosition & key).fetch("body_part")
elif not isinstance(body_parts, list):
    body_parts = list(body_parts)


In [ ]:
# Construct Pandas MultiIndex DataFrame
df = None
for body_part in body_parts:
    result_dict = (
        facemap_pose.BodyPartPosition
        & {"body_part": body_part}
        & {"recording_id": key["recording_id"]}
        & {"session_id": key["session_id"]}
    ).fetch("x_pos", "y_pos", "likelihood", as_dict=True)[0]
    x_pos = result_dict["x_pos"].tolist()
    y_pos = result_dict["y_pos"].tolist()
    likelihood = result_dict["likelihood"].tolist()
    a = np.vstack((x_pos, y_pos, likelihood))
    a = a.T
    pdindex = pd.MultiIndex.from_product(
        [[model_name], [body_part], ["x", "y", "likelihood"]],
        names=["model", "bodyparts", "coords"],
    )
    frame = pd.DataFrame(a, columns=pdindex, index=range(0, a.shape[0]))
    df = pd.concat([df, frame], axis=1)
df

In [ ]:
# Or can use the built in function get_trajectory which also constructs this Pandas MultiIndex DataFrame
# df=facemap_pose.FacemapPoseEstimation.get_trajectory(pose_estimation_key)
# df

In [ ]:
df_xy = df.iloc[:,df.columns.get_level_values(2).isin(["x","y"])]['facemap_model_state.pt']
df_xy.mean()

Plot coordinates across time for each body part

In [ ]:
df_xy.plot().legend(loc='best', prop={'size': 5})

In [ ]:
df_flat = df_xy.copy()
df_flat.columns = df_flat.columns.map('_'.join)

Plot Trace Overlays of each body part across time

In [ ]:
import matplotlib.pyplot as plt 

fig,ax=plt.subplots(2,2)
fig.set_figwidth(20)
fig.set_figheight(15)

df_flat.plot(x='eye(front)_x',y='eye(front)_y',ax=ax[0, 0])
df_flat.plot(x='eye(back)_x',y='eye(back)_y',ax=ax[0, 0])
df_flat.plot(x='eye(bottom)_x',y='eye(bottom)_y',ax=ax[0, 0])

df_flat.plot(x='nose(tip)_x',y='nose(tip)_y', ax=ax[1, 0])
df_flat.plot(x='nose(bottom)_x',y='nose(bottom)_y', ax=ax[1, 0])
df_flat.plot(x='nose(r)_x',y='nose(r)_y', ax=ax[1, 0])
df_flat.plot(x='nosebridge_x',y='nosebridge_y', ax=ax[1, 0])

df_flat.plot(x='mouth_x',y='mouth_y', ax=ax[0, 1])
df_flat.plot(x='lowerlip_x',y='lowerlip_y', ax=ax[0, 1])
df_flat.plot(x='paw_x',y='paw_y', ax=ax[0, 1])

df_flat.plot(x='whisker(I)_x',y='whisker(I)_y', ax=ax[1, 1])
df_flat.plot(x='whisker(II)_x',y='whisker(II)_y', ax=ax[1, 1])
df_flat.plot(x='whisker(II)_x',y='whisker(II)_y', ax=ax[1, 1])
